In [1]:
from qiskit import IBMQ
from qiskit.opflow.primitive_ops import PauliOp, PauliSumOp
from qiskit.aqua.operators.legacy import WeightedPauliOperator           
from qiskit.quantum_info import Pauli
from qiskit.algorithms import VQE
import numpy as np

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 

In [3]:
backend = provider.get_backend('ibmq_lima')
defaults = backend.defaults()

In [4]:
conexions = [ indx for indx in defaults.instruction_schedule_map.qubits_with_instruction('cx') if indx[0]<indx[1]  ]
conexions

[(0, 1), (1, 2), (1, 3), (3, 4)]

In [5]:
#Hamiltonian 
def HeisenbergHamiltonian( J=1, H=1, num_qubits=2, neighbours=[(0,1)] ):
    """
    Qiskit operator of the 3-D Heisemberg Hamiltonian of a lattice of spins.
    
    H = - J Σ_j ( X_j X_{j+1} + Y_j Y_{j+1} + Z_j Z_{j+1} ) - H Σ_j Z_j
    
    input:
        J          : Real. Coupling constant.
        H          : Real. External magnetic field.
        num_qubits : Integer. Number of qubits.
        neighbours : List of tuples. Coupling between the spins.
    output:
        Hamiltonian : BaseOperator of Qiskit. Heisenberg Hamiltonian of the system.
    """
    num_op = num_qubits + 3*len(neighbours)
    Hamiltonian_op_x = []    
    Hamiltonian_op_z = []  
    Hamiltonian_coef = num_qubits*[-H] + num_op*[-J]
    
    for idx in range(num_qubits):
        op_x = np.zeros( num_qubits )
        op_z = np.zeros( num_qubits )
        op_z[idx] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
    
    for idx in neighbours:
        op_x = np.zeros( num_qubits )
        op_z = np.zeros( num_qubits )
        op_x[idx[0]] = 1
        op_x[idx[1]] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )
        op_z[idx[0]] = 1
        op_z[idx[1]] = 1
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
        op_x[idx[0]] = 0
        op_x[idx[1]] = 0
        Hamiltonian_op_x.append( op_x.copy() )
        Hamiltonian_op_z.append( op_z.copy() )        
        
    Hamiltonian = WeightedPauliOperator( 
        [ [Hamiltonian_coef[j], Pauli( ( Hamiltonian_op_z[j], Hamiltonian_op_x[j] )) ] 
         for j in range(num_op) ] )
    return Hamiltonian

QubitOp = HeisenbergHamiltonian( num_qubits=4, neighbours=[(0,1),(1,2),(2,3),(3,0)] )

C:\Users\Luciano\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>


In [6]:
print( QubitOp.print_details() )

IIIZ	(-1+0j)
IIZI	(-1+0j)
IZII	(-1+0j)
ZIII	(-1+0j)
IIXX	(-1+0j)
IIYY	(-1+0j)
IIZZ	(-1+0j)
IXXI	(-1+0j)
IYYI	(-1+0j)
IZZI	(-1+0j)
XXII	(-1+0j)
YYII	(-1+0j)
ZZII	(-1+0j)
XIIX	(-1+0j)
YIIY	(-1+0j)
ZIIZ	(-1+0j)



In [16]:
def Label2Chain(QubitOp):
    """
    Transform a string of Pauli matrices into a numpy array.
    'I' --> 0
    'X' --> 1
    'Y' --> 2
    'Z' --> 3
    
    input:
        QubitOp : BaseOperator of Qiskit.
    output:
        ops     : narray of the Pauli operators.
                  ops.shape = ( number_of_operators, number_of_qubits )
        coef    : coefficient of each Pauli operator.
    """
    Dict = {'I' : 0,
           'X' : 1,
           'Y' : 2,
           'Z' : 3}
    ops = [[ Dict.get(idx2.to_label()) for idx2 in idx[1]] for idx in QubitOp.paulis ]
    coef = [ idx[0] for idx in QubitOp.paulis ]
    return np.array(ops), coef

In [17]:
PS, coef = Label2Chain(QubitOp)
PS

array([[3, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 3],
       [1, 1, 0, 0],
       [2, 2, 0, 0],
       [3, 3, 0, 0],
       [0, 1, 1, 0],
       [0, 2, 2, 0],
       [0, 3, 3, 0],
       [0, 0, 1, 1],
       [0, 0, 2, 2],
       [0, 0, 3, 3],
       [1, 0, 0, 1],
       [2, 0, 0, 2],
       [3, 0, 0, 3]])

In [9]:
import numpy as np
import networkx as nx
from itertools import permutations
import time

In [10]:
def PauliGraph(PS):#PS==Pauli Strings. AM=Admisible Measurements. WC==Well Connected Qubits. 
    #    If we want to group n Pauli arrays of size N, PS should be a matrix of n rows and N columns,
    # each row representing a Pauli string.
    n=np.size(PS[:,0])
    N=np.size(PS[0,:])
    G = nx.Graph()
    G.add_nodes_from(np.arange(n))
    for i in range(n):
        v_i=PS[i,:]
        for j in range(i+1,n):
            v_j=PS[j,:]
            qubits=np.arange(N)
            noncommonqubits=np.delete(qubits,np.argwhere(v_i==v_j))
            vi=v_i[noncommonqubits]
            vj=v_j[noncommonqubits]
            if (vi*vj!=0).any():
                G.add_edges_from([(i,j)])
    return G


def LDFC(PG):
    SV=sorted(PG.degree, key=lambda x: x[1], reverse=True)#Sorted Vertices by decreasing degree
    n=PG.number_of_nodes()
    aux=list(np.arange(n))
    Color=n*np.ones(n)
    for i in range(n):
        IV=list(list(PG.neighbors(SV[i][0])))#Vertices that are Incompatible with vertex SV[i][0]
        IC=Color[IV]#Colors that are assigned to vertices that are incompatible with vertex SV[i]
        AC=[ elem for elem in aux if elem not in IC]#Available colors for vertex SV[i]
        Color[SV[i][0]]=min(AC)
    MC=int(max(Color))
    Groups=[]
    for i in range(MC+1):
        Groups.append(list(np.argwhere(Color==i)))
    return Color, Groups #Color is an array whose i entry has the color assigned to the i Pauli String.
    #Groups is a list of lists, where the i list comprenhends the arrays assigned to the color i.
        
def TPBgrouping(PS): #PS==Pauli Strings. AM=Admisible Measurements. WC==Well Connected Qubits. 
    #    If we want to group n Pauli arrays of size N, PS should be a matrix of n rows and N columns,
    # each row representing a Pauli string.
    PG=PauliGraph(PS)
    Color, Groups=LDFC(PG)
    N=np.size(PS[0,:])
    Measurements=[]#The list of measurements. Each element will be the total measurement for a certain group. That measurement 
    #will be encoded as an N-array of {0,1,3,4}. 0 will appear in the position k if in the qubit k we can measure with any 
    # basis (will only happen if the k factor of every element of the group is I), 0 will appear in the position k if in the qubit k
    #we can measure with TPBX,...
    for i in range(len(Groups)):
        Mi=[]
        for k in range(N):
            Mi.append(max(PS[Groups[i],k]))
        Measurements.append(Mi)
    return Color, Groups, Measurements


In [19]:
Color, Groups, Measurements = TPBgrouping(PS)

In [20]:
Color

array([2., 2., 2., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2., 0., 1., 2.])

In [21]:
Groups

[[array([4], dtype=int64),
  array([7], dtype=int64),
  array([10], dtype=int64),
  array([13], dtype=int64)],
 [array([5], dtype=int64),
  array([8], dtype=int64),
  array([11], dtype=int64),
  array([14], dtype=int64)],
 [array([0], dtype=int64),
  array([1], dtype=int64),
  array([2], dtype=int64),
  array([3], dtype=int64),
  array([6], dtype=int64),
  array([9], dtype=int64),
  array([12], dtype=int64),
  array([15], dtype=int64)]]

In [22]:
Measurements

[[array([1]), array([1]), array([1]), array([1])],
 [array([2]), array([2]), array([2]), array([2])],
 [array([3]), array([3]), array([3]), array([3])]]

In [23]:
PS

array([[3, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 3],
       [1, 1, 0, 0],
       [2, 2, 0, 0],
       [3, 3, 0, 0],
       [0, 1, 1, 0],
       [0, 2, 2, 0],
       [0, 3, 3, 0],
       [0, 0, 1, 1],
       [0, 0, 2, 2],
       [0, 0, 3, 3],
       [1, 0, 0, 1],
       [2, 0, 0, 2],
       [3, 0, 0, 3]])